使用神经网络进行预测

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [3]:
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/SupplyChain_clean.csv')
data.drop(['Unnamed: 0'], axis=1, inplace=True)
data.head(3)

,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Category Id,Category Name,Customer City,Customer Country,Customer Segment,Customer State,Department Id,Department Name,Market,Order City,Order Country,Order Customer Id,Order Item Discount,Order Item Discount Rate,Order Item Product Price,Order Item Profit Ratio,Order Item Quantity,Sales,Order Region,Order State,Product Name,shipping date (DateOrders),Shipping Mode,Customer Fullname,order_year,order_month,order_weekday,order_hour,order_month_year
0,1,3,4,91.250000,73,40,66,1,0,36,2,4,3,331,70,20755,13.110000,0.04,327.75,0.29,1,327.75,15,475,78,27149,3,1875,2018,1,2,22,36
1,3,5,4,-249.089996,73,40,66,1,0,36,2,4,3,391,69,19492,16.389999,0.05,327.75,-0.80,1,327.75,13,841,78,2209,3,5374,2018,1,5,12,36
2,0,4,4,-247.779999,73,40,452,0,0,5,2,4,3,391,69,19491,18.030001,0.06,327.75,-0.80,1,327.75,13,841,78,1980,3,4426,2018,1,5,12,36


In [4]:
# 数据集构造切分
X_data = data.loc[:, data.columns != 'Sales']
y_data = data['Sales']
X_fraud_train, X_fraud_test, y_fraud_train, y_fraud_test = train_test_split(X_data, y_data, test_size=0.2)

X_late = data.loc[:, data.columns != 'Order Item Quantity']
y_late = data['Order Item Quantity']
X_late_train, X_late_test, y_late_train, y_late_test = train_test_split(X_late, y_late, test_size=0.2)

# 数据规范化
sc = StandardScaler()
X_fraud_train = sc.fit_transform(X_fraud_train)
X_fraud_test = sc.transform(X_fraud_test)
X_late_train = sc.transform(X_late_train)
X_late_test = sc.transform(X_late_test)

In [17]:
# 使用神经网络进行分类
import tensorflow.keras as keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

# BN层，在每个batch上将前一层的激活值重新规范化，使输出的数据呈0-1正态分布
keras.layers.BatchNormalization()
classifier = Sequential()
# 搭建神经网络
classifier.add(Dense(1024, activation='relu', input_dim=X_fraud_train.shape[1]))
classifier.add(Dense(512, activation='relu'))
classifier.add(Dense(128, activation='relu'))
classifier.add(Dense(32, activation='relu'))
classifier.add(Dense(1, activation='relu'))
classifier.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mean_absolute_error'])

Sales 预测

In [18]:
# 模型训练
classifier.fit(X_fraud_train, y_fraud_train, batch_size=1024, epochs=10)

Epoch 1/10
142/142 [==============================] - 1s 6ms/step - loss: 57.2212 - mean_absolute_error: 57.2212
Epoch 2/10
142/142 [==============================] - 1s 6ms/step - loss: 13.8013 - mean_absolute_error: 13.8013
Epoch 3/10
142/142 [==============================] - 1s 6ms/step - loss: 8.4153 - mean_absolute_error: 8.4153
Epoch 4/10
142/142 [==============================] - 1s 6ms/step - loss: 5.8630 - mean_absolute_error: 5.8630
Epoch 5/10
142/142 [==============================] - 1s 6ms/step - loss: 4.5595 - mean_absolute_error: 4.5595
Epoch 6/10
142/142 [==============================] - 1s 7ms/step - loss: 4.0657 - mean_absolute_error: 4.0657
Epoch 7/10
142/142 [==============================] - 1s 7ms/step - loss: 3.4398 - mean_absolute_error: 3.4398
Epoch 8/10
142/142 [==============================] - 1s 6ms/step - loss: 3.2911 - mean_absolute_error: 3.2911
Epoch 9/10
142/142 [==============================] - 1s 7ms/step - loss: 2.8620 - mean_absolute_error: 2.86

In [19]:
train_eval = classifier.evaluate(X_fraud_train,y_fraud_train)
test_eval = classifier.evaluate(X_fraud_test,y_fraud_test)
print('Train Evaluation:', train_eval)
print('Test Evaluation:', test_eval)

1129/1129 [==============================] - 3s 2ms/step - loss: 4.0407 - mean_absolute_error: 4.0407
Train Evaluation: [3.9619040489196777, 3.9619040489196777]
Test Evaluation: [4.040741443634033, 4.040741443634033]


In [20]:
y_fraud_pred = classifier.predict(X_fraud_test, batch_size=512, verbose=1)
y_fraud_pred = np.argmax(y_fraud_pred, axis=1)
print(mean_absolute_error(y_fraud_test, y_fraud_pred))

71/71 [==============================] - 0s 2ms/step
204.52582380212408


quantity 预测

In [ ]:
# 模型训练
classifier.fit(X_late_train, y_late_train, batch_size=1024, epochs=10)

Epoch 1/10
142/142 [==============================] - 1s 7ms/step - loss: 85.8669 - mean_absolute_error: 85.8669
Epoch 2/10
142/142 [==============================] - 1s 7ms/step - loss: 2.1278 - mean_absolute_error: 2.1278
Epoch 3/10
142/142 [==============================] - 1s 7ms/step - loss: 2.1278 - mean_absolute_error: 2.1278
Epoch 4/10
142/142 [==============================] - 1s 7ms/step - loss: 2.1278 - mean_absolute_error: 2.1278
Epoch 5/10
142/142 [==============================] - 1s 7ms/step - loss: 2.1278 - mean_absolute_error: 2.1278
Epoch 6/10
 87/142 [=================>............] - ETA: 0s - loss: 2.1287 - mean_absolute_error: 2.1287

In [12]:
train_eval = classifier.evaluate(X_late_train,y_late_train)
test_eval = classifier.evaluate(X_late_test,y_late_test)
print('Train Evaluation:', train_eval)
print('Test Evaluation:', test_eval)

1129/1129 [==============================] - 3s 3ms/step - loss: 2.1271 - mean_absolute_error: 2.1271
Train Evaluation: [2.127777576446533, 2.127777576446533]
Test Evaluation: [2.127077341079712, 2.127077341079712]


In [15]:
y_late_pred = classifier.predict(X_late_test, batch_size=512, verbose=1)
y_late_pred = np.argmax(y_late_pred, axis=1)
print(mean_absolute_error(y_late_test, y_late_pred))

71/71 [==============================] - 0s 2ms/step
2.127077332151562
